<a href="https://colab.research.google.com/github/ninavdPipple/BERTopic/blob/master/Small_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load packages and arrange right set up

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
!pip install typing_extensions --upgrade
!pip install bertopic
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired
!pip install kora -q
from kora import drive
drive.link_nbs()
!python -m pip install tomotopy
!pip install octis
from octis.dataset.dataset import Dataset
from Evaluation_BERTopic_repo import Trainer, DataLoader, Evaluator
import os
import requests
import json
import numpy as np
!pip install openai
import openai
from openai import AzureOpenAI
from collections import Counter
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_colwidth', -1) # make sure to print entire messages when printed form dataframe
import regex as re


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# Connecting to GPT model

client = AzureOpenAI(
    api_key="139ff4ea58234550972e6f5db98b30a7",
    api_version='2023-05-15',
    azure_endpoint = "https://510-openai.openai.azure.com/"
)

deployment_name='510-chat' #This will correspond to the custom name you chose for your deployment when you deployed a model.

# Define functions for training model

In [18]:
def remove_emojis(text):
  # Define regex expression for emojis
  emoji_pattern = re.compile("["
       u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags=re.UNICODE)

  # Remove emojis
  modified_text = emoji_pattern.sub(r'.', text)
  modified_text = re.sub(r'([\:\<]-?[)(|\\/pP3D])(?:(?=\s))', '.', modified_text)
  modified_text = modified_text.lstrip('.') # Remove dots at start of message that arise as a result

  return modified_text

# Remove sequential interpunction
def remove_repeats(text):
  modified_text=re.sub(r'([^\w])\s*([^\w\s])+', lambda match: '? ' if '?' in match.group() else match.group(2), text)

  return modified_text

In [19]:
import string
from OCTIS_preprocess_repo import Preprocessing
os.chdir(os.path.pardir)

def pre_process_data(raw_data, pre_process, reformulations_path = None):

  # Distinguish between reformulations and normal messages to not overwrite data
  if reformulations_path:
    path = reformulations_path
  else:
    path = f'/content/drive/MyDrive/2024'

  # Pre-process data if still needed
  if os.path.exists(f'{path}/ukraine_{pre_process}'):
    # Obtain existing pre-processed dataset
    dataset = Dataset('ukraine')
    dataset.load_custom_dataset_from_folder(f'{path}/ukraine_{pre_process}')
  else:
    # Save required data in required format
    raw_data['low'] =  raw_data['low'].apply(lambda x: re.sub(r'\n',' ', str(x)))     # Remove enters within message to process messages correctly
    raw_data['low'].to_csv(f'{path}/documents_stripped_low.csv', index=False,  encoding="utf-8-sig")
    raw_data['label'].to_csv(f'{path}/labels.csv', index=False, encoding="utf-8-sig")

    # Remove smileys and sequential punctuation for medium pre-processing
    if pre_process == 'medium':
      raw_data['medium'] = raw_data['low'].apply(remove_emojis)
      raw_data['medium'] = raw_data['medium'].apply(remove_repeats)
      raw_data['medium'].to_csv(f'{path}/documents_stripped_medium.csv', index=False, encoding="utf-8-sig")

    # Make reference to the right documents
    docs = pre_process

    # Initialize pre-processing
    if pre_process == 'heavy':
      preprocessor = Preprocessing(lowercase=True,
          remove_punctuation=True,
          punctuation=string.punctuation,
          remove_numbers=False,
          lemmatize=False,
          language="english",
          split=False,
          verbose=False,
          save_original_indexes=True,
          remove_stopwords_spacy=False,)
      docs = 'medium' # Refer to the right documents
    elif pre_process == 'full':
      preprocessor = Preprocessing(lowercase=True,
                remove_punctuation=True,
                punctuation=string.punctuation,
                remove_numbers=True,
                lemmatize=False,
                language="english",
                split=False,
                verbose=False,
                save_original_indexes=True,
                remove_stopwords_spacy=False,)
      docs = 'medium' # Refer to the right documents
    elif pre_process == 'extra':
      preprocessor = Preprocessing(lowercase=True,
          remove_punctuation=True,
          punctuation=string.punctuation,
          remove_numbers=True,
          lemmatize=True,
          language="english",
          split=False,
          verbose=False,
          save_original_indexes=True,
          remove_stopwords_spacy=False,)
      docs = 'medium' # Refer to the right documents
    else:
      preprocessor = Preprocessing(lowercase=True,
                      remove_punctuation=False,
                      punctuation=string.punctuation,
                      remove_numbers=False,
                      lemmatize=False,
                      language="english",
                      split=False,
                      verbose=True,
                      save_original_indexes=True,
                      remove_stopwords_spacy=False,)
      docs = 'medium' # Refer to the right documents

    # Pre-process
    dataset = preprocessor.preprocess_dataset(documents_path=f'{path}/documents_stripped_{docs}.csv', labels_path=f'{path}/labels.csv')

    # Save the pre-processed dataset
    dataset.save(f'{path}/ukraine_{pre_process}')

  return dataset

In [20]:
from sentence_transformers import SentenceTransformer

def embed_messages(embedding_model, dataset, model):

  # Prepare data
  data = [" ".join(words) for words in dataset.get_corpus()]

  # Calculate embeddings
  embeddings = embedding_model.encode(data, show_progress_bar=True)

  # Prepare data
  data = [" ".join(words) for words in dataset.get_corpus()]

  # Calculate embeddings
  embeddings = embedding_model.encode(data, show_progress_bar=True)

  # Save embeddings
  with open(f'/content/drive/MyDrive/2024/{model}/embeddings.npy', 'wb') as f:
      np.save(f, embeddings)

  return embeddings

In [21]:
def train_model(raw_data, pre_process, embed, model, word2vec_model):

  # Pre-process data
  dataset = pre_process_data(raw_data, pre_process)

  # Embed data
    # Pick embedder
  if embed == 'default':
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    embedding_model.max_seq_length = 512
  elif embed == 'advanced':
    embedding_model = SentenceTransformer('jamesgpt1/sf_model_e5')
  embeddings = embed_messages(embedding_model, dataset, model)

  params = {
    'embedding_model': embedding_model,
    'verbose': True,
    'umap_model': UMAP(n_neighbors=15, n_components=5, min_dist=0.0, low_memory = False, metric='cosine',random_state=1), # Fix UMAP model with random state
    'hdbscan_model': HDBSCAN(min_cluster_size=10, prediction_data=True, metric='euclidean', cluster_selection_method='eom', gen_min_span_tree=True), # min cluster size by default 10
    'vectorizer_model': CountVectorizer(stop_words="english", ngram_range=(1, 2)), # default only ngram (1,1)
    # 'representation_model': KeyBERTInspired(), # strongly influences measures depending on topic words
  }

  trainer = Trainer(dataset=f'/content/drive/MyDrive/2024/ukraine_{pre_process}',
                  model_name="BERTopic",
                  params=params,
                  bt_embeddings=embeddings,
                  custom_dataset=True,
                  verbose=False,
                  word2vec_model=word2vec_model,
                  semi_supervised=False,
                  labels = raw_data['label_num'])
  results, topic_model = trainer.train(save=f"/content/drive/MyDrive/2024/{model}/train_results")


  # Save model
  topic_model.save(f'/content/drive/MyDrive/2024/{model}/topic_model.joblib', serialization="pickle")


# Functions for evaluating the model

### Obtain RRI results

In [22]:
from sklearn.metrics.cluster import pair_confusion_matrix

# Calculate RRI

def get_RRI(topics, labels, leave_out = [-1]):
  """
  Caluclates the refinement Rand Index score while leaving out some topics

  Input:
  topics: the refined topics
  labels: the original labeling
  leave_out: a  list of topic numbers to hold out. By default only exclude outliers

  Returns the Refinement Rand Index
  """
  # Exclude the specified topics
  labels = labels[~topics.isin(leave_out)]
  topics = topics[~topics.isin(leave_out)]

  # Measure on refinement; inspired on Rand Index
  contingency = pair_confusion_matrix(labels, topics)
  RRI = (contingency[1][1]+contingency[1][0]+contingency[0][0])/ (contingency[1][1]+contingency[1][0]+contingency[0][0]+contingency[1][1])
  return RRI

### Obtain coherence results

In [23]:
def get_coherence(data, document_info, topic_model, embeddings, dataset, word2vec_model):
  doc_ids = range(len(data))
  documents = pd.DataFrame({"Document": data,
                            "ID": doc_ids,
                            'Class':  document_info['Topic']})
  bertopic_evaluator =  Evaluator(documents, embeddings, topic_model = topic_model, name = 'BERTopic', dataset =dataset, word2vec_model=word2vec_model, verbose = False)
  scores = bertopic_evaluator.score(save = None)
  return scores

### Obtain SRA results




In [24]:
import random
import time

def balanced_subsample(n, raw_data):
    n_categories = raw_data['label'].nunique()
    counts = raw_data.label.value_counts().sort_values()
    sorted_labels = counts.index.tolist()

    n_remaining = n
    subsample = []

    for label, count in counts.items():
        samples_per_category = round(n_remaining/n_categories) # Calculate the new number of samples per category
        if count < samples_per_category:
            sample_size =  count
        else:
            sample_size = samples_per_category

        label_ids = np.where(raw_data['label'] ==  label)[0]
        sample_ids = random.sample(label_ids.tolist(), sample_size)
        subsample.extend(sample_ids)
        n_remaining -= sample_size
        n_categories -= 1

    return raw_data.low.iloc[subsample]

def create_reformulations(raw_data, n, path):
  # Returns and saves reformulated messages
  prompt_format = """
  I have a Telegram message that looks as follows: [MESSAGE]

  Based on the information above, please rewrite the Telegram message while keeping the pronoun the same in the following format:
  Message: <reformulation>
  """
  subsample = balanced_subsample(n, raw_data)
  reformulations = pd.DataFrame({'Original': subsample.to_numpy(), 'Reformulation': np.repeat('', n)})
  reformulations.set_index(np.array(subsample.index), inplace=True)

  delay_in_seconds=1 # Delay in between subsequent prompts to avoid RateLimitError of OpenAI


  for i, message in enumerate(reformulations.Original):
    prompt = prompt_format.replace('[MESSAGE]', message)
    messages = [{"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}]
    kwargs = {"model": deployment_name, "messages": messages}

    response = client.chat.completions.create(**kwargs)
    reformulation = response.choices[0].message.content.strip().replace("Message: ", "")

    # Remove remaining introduction
    reformulation = reformulation.replace("<Reformulation>","").strip()
    reformulation = reformulation.replace("<reformulation>","")
    reformulation = reformulation.replace("<Message>","")
    reformulation = reformulation.replace("<message>","")
    reformulation = reformulation.replace("Reformulation:","")
    reformulation = reformulation.replace("reformulation:","")
    reformulation = reformulation.replace("Message:","")
    reformulation = reformulation.replace("message:","")

    reformulations.Reformulation.iloc[i] = reformulation
    time.sleep(delay_in_seconds)

  # Store because of expensive recalculation
  if not os.path.exists(f'{path}'):
    os.makedirs(f'{path}')
  reformulations.to_csv(path, index=True, encoding="utf-8-sig")

  return reformulations

from sklearn.metrics.pairwise import cosine_similarity

def calculate_SRA(topic_model, document_info, pre_process, reformulations = None, raw_data = None, n=20, save_to ='/content/drive/MyDrive/2024/reformulations'):
  # Either input reformulations or raw_data and n
  # Calculates the SRA score

  # Obtain reformulated messages
  if reformulations is None:
    refomulations = create_reformulations(raw_data, n, path = f'{save_to}.csv')

  # Pre-process reformulations
  test_data = reformulations.rename(columns ={'Reformulation':'low'})
  test_data['label'] = np.repeat('', n) # Create fake label data in order to be able to apply pre-processing
  test_data = pre_process_data(test_data, pre_process, reformulations_path = save_to)
  test_data = test_data.get_corpus()
  test_data = [" ".join(words) for words in test_data]

  # Obtain results for reformulations
  reformulation_topics, reformulation_dist = topic_model.transform(test_data) # OUPTPUT DIFFERS WITH WHICH OTHER MESSAGES A REFORMULATION IS PROCESSED

  # Calculate SRA
  cos_sim = np.diag(cosine_similarity(reformulation_dist, topic_model.probabilities_[reformulations.index]))
  SRA = np.mean(cos_sim)

  # Backhand calculations
  SRA_info = reformulations.copy()
  SRA_info['Original topic'] = document_info.Topic.iloc[reformulations.index]
  SRA_info['Reformulation topic']=  reformulation_topics
  SRA_info['cos_sim'] = cos_sim

  return SRA, SRA_info

def SRA_results(raw_data, document_info, topic_model, pre_process, repeats =1, existig_reformulations = True, n=20): # Boolean if saved reformulations can be used
  SRAs = []

  for run in range(repeats):
    path  = f'/content/drive/MyDrive/2024/reformulations/{run}'
    if existig_reformulations:
      reformulations = pd.read_csv(f'{path}.csv', index_col =0)
      SRA, SRA_info = calculate_SRA(topic_model, document_info, pre_process, reformulations = reformulations, save_to = path)
    else:
      SRA, SRA_info = calculate_SRA(topic_model, document_info, pre_process, raw_data = raw_data, n=n, save_to = path)
    SRAs.append(SRA)

  # Show insights of last run
  return SRAs, SRA_info

# Load model

In [25]:
# Load data
raw_data = pd.read_csv('/content/drive/MyDrive/2024/raw_data.csv', encoding="utf-8-sig")

In [26]:
# Load word2vec model
from gensim.models import KeyedVectors
word2vec_model = KeyedVectors.load('/content/drive/MyDrive/2024/word2vec_model.kv')

In [27]:
def run_pipeline(pre_process,embed, raw_data, word2vec_model, existig_reformulations = True):
  # Pick model
  if pre_process == 'low' and embed == 'default':
    model = 'LD'
  elif pre_process == 'medium' and embed == 'default':
    model = 'MD'
  elif pre_process == 'low' and embed == 'advanced':
    model = 'LA'
  elif pre_process == 'medium' and embed == 'advanced':
    model = 'MA'
  elif pre_process == 'heavy' and embed == 'default':
    model = 'HD'
  elif pre_process == 'full' and embed == 'default':
    model = 'FD'
  elif pre_process == 'extra' and embed == 'default':
    model = 'ED'

  # Check if model has been trained already
  new_model = not os.path.exists(f"/content/drive/MyDrive/2024/{model}")
  if new_model:
    os.makedirs(f"/content/drive/MyDrive/2024/{model}")
    train_model(raw_data, pre_process = pre_process, embed = embed, model = model, word2vec_model=word2vec_model)
    print('Finished training '+model)
  else:
    print('Loading '+model)

  # Load pre-processed dataset
  dataset = Dataset('ukraine')
  dataset.load_custom_dataset_from_folder(f'/content/drive/MyDrive/2024/ukraine_{pre_process}')
  list_data = dataset.get_corpus()
  data = [" ".join(words) for words in list_data]

  # Load embeddings
  with open(f'/content/drive/MyDrive/2024/{model}/embeddings.npy', 'rb') as f:
    embeddings = np.load(f)

  # Load model
  topic_model = BERTopic.load(f'/content/drive/MyDrive/2024/{model}/topic_model.joblib')

  # Obtain results
  document_info = topic_model.get_document_info(data)

  # Get metrics
  RRI = get_RRI(document_info['Topic'], raw_data['label'])
  coherence_scores = get_coherence(data, document_info, topic_model, embeddings,dataset, word2vec_model)
  DBCV = topic_model.hdbscan_model.relative_validity_

  # Save the values of all metrics
  scores = coherence_scores
  scores['RRI'] = RRI
  scores[ 'DBCV'] = DBCV
  SRAs, SRA_info = SRA_results(raw_data, document_info, topic_model, pre_process, repeats =1, existig_reformulations = existig_reformulations, n=20)
  scores['SRA'] = np.mean(SRAs)
  with open(f"/content/drive/MyDrive/2024/{model}/results.json", "w") as f:
      json.dump(scores, f)
  print(scores)

  return topic_model, document_info, SRAs, SRA_info


In [29]:
create_embeddings = False

for pre_process in ['extra']:
  for embed in ['default']:
    run_pipeline(pre_process,embed, raw_data, word2vec_model, existig_reformulations = not create_embeddings)
    create_embeddings = False # At most once needed to create the embeddings for testing multiple models

100%|██████████| 4522/4522 [01:22<00:00, 54.71it/s]


created vocab
6497


Batches:   0%|          | 0/142 [00:00<?, ?it/s]

Batches:   0%|          | 0/142 [00:00<?, ?it/s]

2024-02-21 14:31:54,152 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-02-21 14:32:06,499 - BERTopic - Dimensionality - Completed ✓
2024-02-21 14:32:06,501 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-02-21 14:32:08,376 - BERTopic - Cluster - Completed ✓
2024-02-21 14:32:08,388 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-02-21 14:32:09,170 - BERTopic - Representation - Completed ✓
2024-02-21 14:32:13,633 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


Finished training ED


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-02-21 14:32:28,276 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2024-02-21 14:32:30,355 - BERTopic - Dimensionality - Completed ✓
2024-02-21 14:32:30,359 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2024-02-21 14:32:30,377 - BERTopic - Probabilities - Start calculation of probabilities with HDBSCAN
2024-02-21 14:32:30,456 - BERTopic - Probabilities - Completed ✓
2024-02-21 14:32:30,462 - BERTopic - Cluster - Completed ✓


{'npmi': 0.13277869506774295, 'we': 0.1797294633687982, 'diversity': 0.41847826086956524, 'RRI': 0.013209062100724072, 'DBCV': 0.19728483513805795, 'SRA': 0.8354165046132179}


# Load data in right format

In [ ]:
# # Load data
# raw_data = pd.read_excel('/content/drive/MyDrive/messages_2023_list.xlsx')

# # Remove enters in label names
# raw_data['label'] =  raw_data['label'].apply(lambda x: re.sub(r'\n','', str(x)))

# # Create class numbers for each label for semi-supervised learning
# label_to_num = {k: v for v, k in enumerate(pd.Series(raw_data['label']).unique())}
# raw_data['label_num'] = raw_data['label'].map(label_to_num)

# # Remove urls
# raw_data['text'] = raw_data['text'].apply(lambda x: re.sub(r'\(https[^\)]*\)',' <URL> ', str(x))) # Remove based on brackets
# raw_data['text'] = raw_data['text'].apply(lambda x: re.sub(r'https[\S]*','<URL>', str(x))) # Remove based on followed by a space or enter

# # Remove any remaining Ukrainian text
# raw_data['low'] =  raw_data['text'].apply(lambda x: re.sub(r'\p{IsCyrillic}','', str(x)))

# # Save required data
# raw_data.to_csv('/content/drive/MyDrive/2024/raw_data.csv', index=False, encoding="utf-8-sig")

In [ ]:
# from IPython.display import display, HTML

# def pretty_print(df):
#   # Prints DataFrames in a neat format
#   return display( HTML( df.to_html().replace("\\n","<br>")))

# # Print examples from pre-processing

# raw_data_examples = pd.read_excel('/content/drive/MyDrive/messages_2023_list_old.xlsx')

# examples = raw_data_examples[['text', 'label']].iloc[[565,1448,456]]

# pretty_print(examples)

# # Remove any remaining Ukrainian text
# raw_data_examples['low'] =  raw_data_examples['text'].apply(lambda x: re.sub(r'\p{IsCyrillic}','', str(x)))

# pretty_print(pd.DataFrame(raw_data_examples[['text', 'low']].iloc[2266]).transpose().rename(columns={'text':'Original', 'low': 'Pre-processed'}))

In [ ]:
# import gensim.downloader as api

# word2vec_model = api.load('word2vec-google-news-300')
# word2vec_model.save('/content/drive/MyDrive/2024/word2vec_model.kv')

# Inspect data

In [ ]:
# Analyze data

# # Overview of categories
# plt.figure().set_figheight(7)
# sns.countplot(y = 'label', data=raw_data, order = raw_data['label'].value_counts().index)
# plt.title('Messages per category')
# plt.xlabel('Amount of messages')
# plt.ylabel('Label')
# plt.show()


In [ ]:
# #%%
# words = [re.sub('[^a-z]','',word.lower()) for message in list_data for word in message] # Remove punctuation and digits and lowercase all words
# no_stops = [word for word in words if word not in stopwords.words('english')] # Exclude stopwords
# counts = Counter(no_stops)
# print(counts.most_common(50))

In [ ]:
# # Analyse lenght of messages
# lengths = [len(message) for message in list_data]
# print('Max lenght:', max(lengths))
# print('Min lenght:', min(lengths))
# longest_messages = (-np.array(lengths)).argsort()[:10]
# sns.histplot(lengths)
# plt.title('Distribution of message sizes')
# plt.xlabel('Number of words')
# plt.ylabel('Amount of messages')
# plt.xticks(np.arange(0,550,50))
# plt.show()

In [ ]:
import torch

tokenizer = embedding_model.tokenizer

# Tokenize sentences
tokenized_messages = tokenizer(data, padding=True, truncation=False, return_tensors='pt')

# Check how many messages cut-off by truncation of SBERT

num_tokens = torch.sum(tokenized_messages['input_ids'] > 0, dim=1)
# sns.histplot(num_tokens)
num_tokens = pd.Series(num_tokens)
bins = [0,256,512,1000] # Default truncation is after 256 and can be adjusted to max 512
print(num_tokens.value_counts(bins=bins))

# Truncation has barely any result. Also because urls blanked and digits removed

NameError: name 'embedding_model' is not defined

In [ ]:
# documents = pd.DataFrame({"Document": data,
#                           "ID": doc_ids,
#                           'Class':  raw_data['label']})

# baseline_evaluator =  Evaluator(documents, embeddings, topic_model = BERTopic(umap_model= umap_model), name = 'Original labeling', dataset =dataset, word2vec_model=word2vec_model)

# baseline_evaluator.score(save= f"/content/drive/MyDrive/2024/baseline_results")


In [ ]:
# # Remove multiple occurences of the same character
# # from Tweettokenizer
#     pattern = re.compile(r"(.)\1{2,}")
#     return pattern.sub(r"\1\1\1", text)

In [ ]:
def check_cluster(num):
  # Check specific cluster
  return (document_info['Document'][document_info['Topic']== num])

In [ ]:
# # Briefly inspect Introdcution effect

# example_reformulations = pd.read_csv('/content/drive/MyDrive/2024/reformulations.csv', index_col =0)
# SRA, SRA_info = calculate_SRA(topic_model,reformulations = example_reformulations)
# print(SRA_info.cos_sim.iloc[-1])

# cleaned =  example_reformulations.copy()
# alternative = cleaned.Reformulation.iloc[-1].replace('<Message>\n', '')
# cleaned.Reformulation.iloc[-1] = alternative
# SRA, SRA_info = calculate_SRA(topic_model,reformulations = cleaned)
# SRA_info.cos_sim.iloc[-1]

In [ ]:
print('Found topic for reformulation:')
print(topic_model.get_topic_info()[['Topic', 'Representation']].iloc[[5+1]].to_string())
print(check_cluster(5))

print('\n Reformulation in outlier: ')
print(topic_model.get_topic_info()[['Topic', 'Representation']].iloc[[4+1]].to_string())
print(check_cluster(4))

print('\n Confuse topics:')
print(topic_model.get_topic_info()[['Topic', 'Representation']].iloc[[2+1,56+1]].to_string())

In [ ]:
#%%

# Analyse results
print(topic_model.get_topic_info())
print(document_info)

In [ ]:
# from OpenAI_BERTopic_repo import OpenAI

# summarization_prompt = """
# This is a list of texts where each collection of texts describe a topic. After each collection of texts, a summary that captures the most important content of the texts in the collections is provided.
# ---
# Topic:
# Sample texts from this topic:
# - Traditional diets in most cultures were primarily plant-based with a little meat on top, but with the rise of industrial style meat production and factory farming, meat has become a staple food.
# - Meat, but especially beef, is the worst food in terms of emissions.
# - Eating meat doesn't make you a bad person, not eating meat doesn't make you a good one.

# Keywords: meat beef eat eating emissions steak food health processed chicken
# Summary: Meat has become a bigger component of our diets. Eating meat is harmful for the enviroment.
# ---
# Topic:
# Sample texts from this topic:
# - I have ordered the product weeks ago but it still has not arrived!
# - The website mentions that it only takes a couple of days to deliver but I still have not received mine.
# - I got a message stating that I received the monitor but that is not true!
# - It took a month longer to deliver than was advised...

# Keywords: deliver weeks product shipping long delivery received arrived arrive week
# Summary: Some packages take a long time to get delivered. False claims about the delivery of packages are being made.
# ---
# Topic:
# Sample texts from this topic:
# [DOCUMENTS]
# Keywords: [KEYWORDS]
# Summary:"""

# representation_model= OpenAI(client,model = deployment_name, prompt = summarization_prompt, nr_docs=20, delay_in_seconds=3)
# representation_model.extract_topics(topic_model, documents.rename(columns={'Class':'Topic'}), topic_model.c_tf_idf_, topic_model.topic_representations_)

In [ ]:
# Select cluster to analyse
cluster_num = 36

In [ ]:
from OpenAI_BERTopic_repo import OpenAI

summarization_chat_prompt = """
I have a topic that is described by the following keywords: [KEYWORDS]
In this topic, the following documents are a small but representative subset of all documents in the topic:
[DOCUMENTS]

Based on the information above, please write a summary that captures the most important content of this topic in the following format:
topic: <summary>
"""

representation_model = OpenAI(client, model=deployment_name, chat=True, prompt=summarization_chat_prompt, nr_docs=20, delay_in_seconds=3)
representation_model.extract_topics(topic_model, documents[documents['Class']==cluster_num].rename(columns={'Class':'Topic'}), topic_model.c_tf_idf_.getrow(cluster_num), {cluster_num:topic_model.topic_representations_[cluster_num]})

In [ ]:
print(topic_model.get_topic_freq().head(11))

In [ ]:
# Overview of cluster sizes
sns.barplot(y = 'Count', x = 'Topic', data=topic_model.get_topic_freq().iloc[1:])
plt.show()


In [ ]:
# Check biggest cluster
print(document_info['Document'][document_info['Topic']== 0])

In [ ]:
# Analyze clusters of large text messages

print(raw_data['text'].iloc[longest_messages])
print(document_info['Topic'].iloc[longest_messages])

# Check specific cluster
print('Longest cluster:', document_info['Top_n_words'].iloc[longest_messages[0]])
print(document_info['Document'][document_info['Topic']== document_info['Topic'].iloc[longest_messages[0]]])

In [ ]:
fig = topic_model.visualize_topics()
fig.write_html("inter.html")
fig.show()

In [ ]:
heatmap = topic_model.visualize_heatmap()
heatmap.show()

In [ ]:
# Check specific cluster
print(check_cluster(74))

In [ ]:
#%%

# Check if not focussing on marker tokens
# Could potentionally check with tf-idf score
# Now because of preprocessing punctuation is removed, so markers not this specific anymore

pers_topics = pd.concat([document_info['Topic'],raw_data['text'].str.contains('<PERSON>')],axis=1)
freq_pers = pers_topics.groupby(['Topic'], as_index=False).mean()
sns.barplot(x = 'Topic', y= 'text', data=freq_pers, color='b')
plt.title('<PERSON> marker')
plt.ylabel('Percetage contains <PERSON>')
plt.show()

url_topics = document_info['Topic'][raw_data['text'].str.contains('<URL>')]
sns.histplot(url_topics, binwidth=1)
plt.title('<URL> marker')
plt.show()

In [ ]:
# Get matrix with for every word the importance for that topic and the corresponding words
topic_term_matrix = topic_model.c_tf_idf_.toarray()
words = topic_model.vectorizer_model.get_feature_names_out()

# Get the column corresponding to person
person_col = np.where(words=='person')[0]

# Get for each topic the ranking on how important person is
person_importance = np.where(np.argsort(-topic_term_matrix)==person_col)[1]

# Get the ranking for the topic where it's most important (exclude the outliers)
max_importance = np.min(person_importance[1:])
print(max_importance)

In [ ]:
KK_topics = pd.concat([document_info['Topic'],raw_data['text'].str.contains(' KK ')],axis=1)
freq_KK = KK_topics.groupby(['Topic'], as_index=False).mean()
sns.barplot(x = 'Topic', y= 'text', data=freq_KK, color='b')
plt.title('KK')
plt.ylabel('Percetage contains KK')
plt.show()


In [ ]:
#%%

# Ditribution over topics

topic_distr, _ = topic_model.approximate_distribution(data, window=8, stride=4)
topic_model.visualize_distribution(topic_distr[0], custom_labels=True)

# Maybe nice for long messages

In [ ]:
instance = 3121

# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(data, calculate_tokens=True)

# Visualize the token-level distributions
topic_model.visualize_approximate_distribution(data[instance], topic_token_distr[instance])

In [ ]:
topics_per_class = topic_model.topics_per_class(raw_data['text'], classes=raw_data['label'])

fig = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=None)
# fig.write_html("class.html")
fig.show()

In [ ]:
raw_data[['text', 'label']][document_info.Topic==79]

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(raw_data['text'])
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
# Assign an original label to topics if more than a threshold percentage of the messages in the topic originate from that label

threshold = 0.7
raw_data['topic']= document_info['Top_n_words']
clustered_only = raw_data[document_info['Topic']!=-1] # Exclude outliers

# Count the number of occurences of each label within each topic
label_count = clustered_only[['topic', 'label']].groupby(['topic', 'label']).size().reset_index(name='count')

# Obtain the frequency how much of the topic originated from which label
label_count['freq'] = label_count.groupby(['topic'])['count'].apply(lambda x: x / x.sum()) # DataFrame containing per topic the distribution over the original labels

# Say a topic is a refinement if more that the threshold originated from the same label
clear_refined = label_count[label_count['freq']>=threshold][['label', 'freq', 'topic']]
print(str(len(clear_refined)), ' topics originate clearly from refinement on labeling')
print(clear_refined.groupby(['label'])['label', 'topic'].apply(print))


In [ ]:
# Get topics that are not from a parcticular label

potential_new = label_count[label_count['freq']<threshold][['label', 'freq', 'topic']]
covered_topics = clear_refined['topic'].tolist()

# Exclude refinement topics
potential_new = potential_new[~potential_new['topic'].isin(covered_topics)]

# Order topics based on being least certain to be part of the refinement
potential_new = potential_new.groupby('topic')['freq'].max().reset_index(name='certainty') # Get the assignment score to the label it was most likely to occur
potential_new = potential_new.sort_values('certainty')
print(potential_new)


In [ ]:
document_info['Document'][document_info['Top_n_words'] ==  potential_new['topic'].iloc[0]]

In [ ]:
# from sklearn.metrics import silhouette_score

# # Obtain the umap embeddings
# umap_embeddings = topic_model.umap_model.transform(embeddings)

# # Exclude outliers
# umap_embeddings = umap_embeddings[document_info['Topic']!=-1]
# clustering = document_info['Topic'][document_info['Topic']!=-1]

# # Calculate silhouette score
# print(silhouette_score(umap_embeddings, clustering))
